# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [4]:
!mkdir data

mkdir: data: File exists


In [5]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31287    0 31287    0     0      0      0 --:--:-- --:--:-- --:--:--     0 52849      0 --:--:-- --:--:-- --:--:-- 52939


In [6]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70146    0 70146    0     0  82427      0 --:--:-- --:--:-- --:--:-- 82330


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [1]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [5]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [6]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [7]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [8]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 14, relationships: 70)

We can save and load our knowledge graphs as follows.

In [9]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 14, relationships: 70)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [10]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [11]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


- SingleHopSpecificQuerySynthesizer:
    
    This synthesizer generates specific, fact-based queries that can be answered using information from a single chunk or node of the knowledge graph4. It focuses on clear, straightforward questions that don't require combining information from multiple sources.
- MultiHopAbstractQuerySynthesizer:
    
    This synthesizer creates more complex, abstract queries that require information from multiple nodes in the knowledge graph3. It generates questions that involve higher-level thinking, comparisons, or synthesizing information from different parts of the document.
- MultiHopSpecificQuerySynthesizer:
    
    This synthesizer produces specific queries that require information from multiple nodes in the knowledge graph6. It creates questions that need concrete facts from different parts of the document to be combined for a complete answer.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [12]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What role does Anthropic play in the developme...,[Code may be the best application The ethics o...,Anthropic is one of the organizations that hav...,single_hop_specifc_query_synthesizer
1,Wht r the implicashuns of AGI on the gullibili...,[Based Development As a computer scientist and...,The development of AGI is seen as a potential ...,single_hop_specifc_query_synthesizer
2,What insights did Simon Willison share about t...,[Simon Willison’s Weblog Subscribe Stuff we fi...,Simon Willison highlighted that 2023 was a bre...,single_hop_specifc_query_synthesizer
3,what stable diffusion mean in context of large...,[easy to follow. The rest of the document incl...,The term 'Stable Diffusion' in the context of ...,single_hop_specifc_query_synthesizer
4,Wut are sum of the key issues with Apple Intel...,[Prompt driven app generation is a commodity a...,"Apple Intelligence is described as bad, while ...",single_hop_specifc_query_synthesizer
5,What are the ethical and legal challenges asso...,[<1-hop>\n\nCode may be the best application T...,The ethical and legal challenges associated wi...,multi_hop_abstract_query_synthesizer
6,How does the black box nature of large languag...,[<1-hop>\n\nCode may be the best application T...,The black box nature of large language models ...,multi_hop_abstract_query_synthesizer
7,How has OpenAI influenced the development of A...,[<1-hop>\n\nCode may be the best application T...,OpenAI has played a significant role in the de...,multi_hop_abstract_query_synthesizer
8,What are the key differences between GPT-4 and...,[<1-hop>\n\nreasoning patterns. Another common...,"GPT-4o, unlike its predecessor GPT-4, can run ...",multi_hop_specific_query_synthesizer
9,How did the development of Claude 3 and its su...,[<1-hop>\n\nPrompt driven app generation is a ...,The development of Claude 3 and its subsequent...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [14]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Wht is Stablity AI?,[Code may be the best application The ethics o...,Stability AI is one of the organizations that ...,single_hop_specifc_query_synthesizer
1,What are some challenges associated with using...,[Based Development As a computer scientist and...,Some challenges associated with using ChatGPT ...,single_hop_specifc_query_synthesizer
2,What significant developments occurred with LL...,[Simon Willison’s Weblog Subscribe Stuff we fi...,2023 was the breakthrough year for Large Langu...,single_hop_specifc_query_synthesizer
3,what chatgpt do in 2023?,[easy to follow. The rest of the document incl...,"In 2023, ChatGPT was used to extract data into...",single_hop_specifc_query_synthesizer
4,Why LLMs called black boxes and how they impac...,[<1-hop>\n\nCode may be the best application T...,"LLMs are referred to as black boxes because, d...",multi_hop_abstract_query_synthesizer
5,How do the challenges of understanding and con...,[<1-hop>\n\nCode may be the best application T...,The challenges of understanding and controllin...,multi_hop_abstract_query_synthesizer
6,How do the challenges of using Large Language ...,[<1-hop>\n\nCode may be the best application T...,The challenges of using Large Language Models ...,multi_hop_abstract_query_synthesizer
7,How has the environmental impact of AI evolved...,[<1-hop>\n\nCode may be the best application T...,"The environmental impact of AI, particularly w...",multi_hop_abstract_query_synthesizer
8,What are some key developments in LLMs in 2024...,[<1-hop>\n\nBased Development As a computer sc...,"In 2024, significant developments in Large Lan...",multi_hop_specific_query_synthesizer
9,How does DeepSeek v3 demonstrate advancements ...,[<1-hop>\n\nreasoning patterns. Another common...,"DeepSeek v3, a 685B parameter model, showcases...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [15]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [16]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [17]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [19]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [20]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [21]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [22]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [23]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Finally, we can set-up our RAG LCEL chain!

In [24]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [25]:
rag_chain.invoke({"question" : "What are Agents?"})

'Agents are referred to as "AI systems that can go away and act on your behalf," but the term is considered vague and encompasses numerous definitions. There are two main categories: one that views agents as entities that act on behalf of users, similar to the travel agent model, and another that regards them as LLMs (Large Language Models) with access to tools that can operate in a loop to solve problems. Overall, the concept of agents is still developing, and there are few examples of them running in production.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4o as our evaluation LLM for our base Evaluators.

In [26]:
eval_llm = ChatOpenAI(model="gpt-4o")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [27]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- qa_evaluator: Evaluates the quality and accuracy of the answers provided in the dataset.

- labeled_helpfulness_evaluator: Assesses how helpful the labeled data is for training and evaluating models.

- dope_or_nope_evaluator: Determines whether the generated data is considered good ("dope") or not ("nope") based on certain criteria.

## LangSmith Evaluation

In [28]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'diligent-transport-14' at:
https://smith.langchain.com/o/7acbdd88-ea69-5821-8eee-b1b6853db36b/datasets/79b30131-c34b-44c3-b02e-a8e6e55bcf99/compare?selectedSessions=e4370548-0e70-4e20-a0b2-6eb3084aa6ef




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How did Claude 3 contribute to the advancement...,"Claude 3, developed by Anthropic, contributed ...",None,"Claude 3, developed by Anthropic, significantl...",1,0,1,3.079068,d4297b3c-f6aa-486f-b6a6-680f52759557,f711df75-4ba8-4add-bd1b-6e8e73bde23e
1,How does the development of GPT-4V relate to t...,I don't know.,None,Simon Willison's 2023 review highlights that 2...,0,0,0,1.147025,3c146b8c-fe36-41d3-860f-d038029b9a52,bf29a3ab-1243-45ac-8fad-60af3db3e9de
2,How does DeepSeek v3 demonstrate advancements ...,DeepSeek v3 demonstrates advancements in AI ef...,None,"DeepSeek v3, a 685B parameter model, showcases...",1,1,1,4.212400,9980d94f-1c14-4f0e-9d9c-80901ce4a137,f1681f79-f4d4-4088-b8f0-6083159e68c1
3,What are some key developments in LLMs in 2024...,I don't know.,None,"In 2024, significant developments in Large Lan...",0,0,0,1.881777,e1a0d6dc-0267-4921-a019-0ec3ffa15cfc,4930799a-4b63-49c0-b90b-f9c19cad4d09
4,How has the environmental impact of AI evolved...,The context mentions that the environmental im...,None,"The environmental impact of AI, particularly w...",0,0,0,2.201194,ad261c01-1446-43d7-960a-e6f8cdfdb7e4,6a96a66a-3d0a-413d-91c3-fb87692a474a
5,How do the challenges of using Large Language ...,I don't know.,None,The challenges of using Large Language Models ...,0,0,0,1.018909,2f44e68b-d8cb-49d9-9ecb-19d7459ce81f,34a8c6a9-69cd-476e-9475-623146544ff3
6,How do the challenges of understanding and con...,I don't know.,None,The challenges of understanding and controllin...,0,0,0,0.825765,4a4eaa43-8aeb-436d-b95a-4436898b5daa,342faacb-ff2b-4069-804e-70bc80534cd6
7,Why LLMs called black boxes and how they impac...,"LLMs are called black boxes because, even thou...",None,"LLMs are referred to as black boxes because, d...",0,0,0,2.250533,33d291fb-aeea-41f5-ac10-26d0782f8515,e4dd4ac9-f156-44a6-a961-d16042306cc9
8,what chatgpt do in 2023?,"In 2023, ChatGPT had various features and deve...",None,"In 2023, ChatGPT was used to extract data into...",0,1,0,2.464057,24ad3a74-bbfe-493c-8535-c9c17e99e705,0be0151d-feb5-4f06-a9ee-9679f90858da
9,What significant developments occurred with LL...,"In 2023, the only organization that had releas...",None,2023 was the breakthrough year for Large Langu...,1,1,0,2.571336,be2d37ef-a792-45d1-903a-5fe6c19d2203,e698de27-efd6-467f-b3e9-cbcfa3b97038


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [29]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [30]:
rag_documents = docs

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

If chunks are too small, they may lose important context. If they're too large, the retrieval process becomes less efficient and may include irrelevant information. With too much context, models are susceptible to lost in the middle problems.

In [32]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

Embedding models map queries and documents into a shared vector space, enabling RAG systems to quickly identify and retrieve the most relevant information. Different embedding models may capture different aspects of the data, leading to variations in the accuracy of downstream tasks. Embedding models with larger dimensions or more parameters will consume more memory, which can affect the scalability of the application.


In [33]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [34]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [36]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [37]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Yo, here’s the lowdown on Agents: they\'re these AI systems folks are buzzing about that are supposed to act on your behalf—kind of like a digital travel agent. But the term is super vague, and it means different things to different people. Some think of them as LLMs running tools to solve problems, while others just see them as assistants doing stuff for you. At the end of the day, they still feel like something that\'s "coming soon," and there ain\'t many real-life examples of them kicking it in production yet. The struggle with gullibility is real too, making it tough for these agents to actually deliver without a solid grasp on what\'s true and what\'s not. So, yeah, Agents are still a bit of a mystery!'

Finally, we can evaluate the new chain on the same test set!

In [38]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'warm-line-97' at:
https://smith.langchain.com/o/7acbdd88-ea69-5821-8eee-b1b6853db36b/datasets/79b30131-c34b-44c3-b02e-a8e6e55bcf99/compare?selectedSessions=5542fa8e-38ae-43c0-a8f5-dcb89ec1b446




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How did Claude 3 contribute to the advancement...,Claude 3 kicked things up a notch in the AI ga...,None,"Claude 3, developed by Anthropic, significantl...",1,0,1,3.548389,d4297b3c-f6aa-486f-b6a6-680f52759557,728a3f3a-5aa1-409f-a061-0aba1aa23f5b
1,How does the development of GPT-4V relate to t...,I don't know.,None,Simon Willison's 2023 review highlights that 2...,0,0,0,1.364864,3c146b8c-fe36-41d3-860f-d038029b9a52,24d92c7c-a187-4cb4-8aaf-e9ee725f6109
2,How does DeepSeek v3 demonstrate advancements ...,"Yo, check this out! DeepSeek v3 is flexing som...",None,"DeepSeek v3, a 685B parameter model, showcases...",1,1,1,7.885197,9980d94f-1c14-4f0e-9d9c-80901ce4a137,220bacde-b356-4cc5-8cd9-bb2b768cf624
3,What are some key developments in LLMs in 2024...,I don't know.,None,"In 2024, significant developments in Large Lan...",0,0,0,1.716030,e1a0d6dc-0267-4921-a019-0ec3ffa15cfc,f68a5e71-1003-4206-81c3-a74143b8534a
4,How has the environmental impact of AI evolved...,"Yo, check it out! The environmental impact of ...",None,"The environmental impact of AI, particularly w...",1,0,1,4.445313,ad261c01-1446-43d7-960a-e6f8cdfdb7e4,8b712901-775a-4e6e-8738-9df3b4978842
5,How do the challenges of using Large Language ...,"Yo, check it out! The challenges of using Larg...",None,The challenges of using Large Language Models ...,1,0,1,5.295338,2f44e68b-d8cb-49d9-9ecb-19d7459ce81f,3c192439-282d-4daf-b2c4-8bab911e5630
6,How do the challenges of understanding and con...,"Yo, the challenges of understanding and contro...",None,The challenges of understanding and controllin...,1,0,1,9.071286,4a4eaa43-8aeb-436d-b95a-4436898b5daa,91d566f6-8441-44f9-aebb-3d6651a0b05b
7,Why LLMs called black boxes and how they impac...,I don't know.,None,"LLMs are referred to as black boxes because, d...",0,0,0,1.438649,33d291fb-aeea-41f5-ac10-26d0782f8515,5eb6908c-9d71-4c64-9a87-2ab0d22b068e
8,what chatgpt do in 2023?,"In 2023, ChatGPT got some slick upgrades, like...",None,"In 2023, ChatGPT was used to extract data into...",0,0,1,3.171982,24ad3a74-bbfe-493c-8535-c9c17e99e705,8b15a30c-2cc9-48f5-8556-f4abf0ec03e8
9,What significant developments occurred with LL...,I don't know.,None,2023 was the breakthrough year for Large Langu...,0,0,0,1.727102,be2d37ef-a792-45d1-903a-5fe6c19d2203,f24975dc-d394-4612-91f7-2abbd650e787


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

![screenshot1](comparison.png)

![screenshot2](comparision2.png)

![screenshot3](comparison3.png)

The second chain:
- Includes a "dope" prompt augmentation
- Uses larger chunks
- Improves the retriever model to: `text-embedding-3-large`

There were more dopeness and correctness evaluations in the second chain (warm-line-97) due to the prompt augmentation. Improved correctness could be from the larger embedding model providing more accurate representations of the input text and larger chunks for added context. 

It's interesting how few responses were 'helpful' for either chain. Most of the evaluation reasoning is, "while the submission answers the question to some extent, it does not fully capture the critical components of helpfulness as defined by the completeness and accuracy of the reference answer."



